In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Define transformations for the training and testing sets
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the ResNet18 model
model = resnet18(num_classes=10)
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Define the training loop
def train(epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 100 == 99:  # Print every 100 mini-batches
            print(f"[Epoch {epoch+1}, Batch {batch_idx+1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

# Define the testing loop
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    print(f"Accuracy: {100.0 * correct / total:.2f}%")

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    train(epoch)
    test()

# Save the trained model
torch.save(model.state_dict(), 'resnet18_cifar10.pth')


Files already downloaded and verified
Files already downloaded and verified
[Epoch 1, Batch 100] loss: 3.133
[Epoch 1, Batch 200] loss: 1.974
[Epoch 1, Batch 300] loss: 1.772
Accuracy: 41.65%
[Epoch 2, Batch 100] loss: 1.570
[Epoch 2, Batch 200] loss: 1.534
[Epoch 2, Batch 300] loss: 1.480
Accuracy: 51.60%
[Epoch 3, Batch 100] loss: 1.355
[Epoch 3, Batch 200] loss: 1.346
[Epoch 3, Batch 300] loss: 1.307
Accuracy: 55.85%
[Epoch 4, Batch 100] loss: 1.214
[Epoch 4, Batch 200] loss: 1.185
[Epoch 4, Batch 300] loss: 1.142
Accuracy: 62.27%
[Epoch 5, Batch 100] loss: 1.088
[Epoch 5, Batch 200] loss: 1.079
[Epoch 5, Batch 300] loss: 1.057
Accuracy: 59.83%
[Epoch 6, Batch 100] loss: 1.006
[Epoch 6, Batch 200] loss: 0.999
[Epoch 6, Batch 300] loss: 0.996
Accuracy: 65.61%
[Epoch 7, Batch 100] loss: 0.950
[Epoch 7, Batch 200] loss: 0.939
[Epoch 7, Batch 300] loss: 0.928
Accuracy: 67.31%
[Epoch 8, Batch 100] loss: 0.892
[Epoch 8, Batch 200] loss: 0.905
[Epoch 8, Batch 300] loss: 0.878
Accuracy: 69.

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import numpy as np

# Define the same transformations used for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 training dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=False, num_workers=2)

# Load the trained ResNet18 model
model = resnet18(num_classes=10)
model.load_state_dict(torch.load('resnet18_cifar10.pth'))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Modify the model to output the embedding from the last layer
class ResNet18_Embedding(nn.Module):
    def __init__(self, original_model):
        super(ResNet18_Embedding, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return x

embedding_model = ResNet18_Embedding(model)
embedding_model = embedding_model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Extract embeddings for the entire training set
def extract_embeddings(dataloader, model):
    embeddings = []
    labels = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(inputs)
            embeddings.append(outputs.cpu())
            labels.append(targets.cpu())
    embeddings = torch.cat(embeddings)
    labels = torch.cat(labels)
    return embeddings, labels

train_embeddings, train_labels = extract_embeddings(trainloader, embedding_model)

# Convert to NumPy arrays and save
train_embeddings_np = train_embeddings.numpy()
train_labels_np = train_labels.numpy()

Files already downloaded and verified


In [3]:
print(train_embeddings_np.shape)
print(train_labels_np.shape)

(50000, 512)
(50000,)


In [ ]:
np.save("../datasets/cifar10_data/cifar10_embedding.npy", train_embeddings_np)
np.save("../datasets/cifar10_data/cifar10_labels.npy", train_labels_np)